# Modelo

La optimizacion la hago con el dataset undersampleado porque sino tarda una eternidad.

En el proximo script entreno ahi si con la totalidad de los datos.

In [1]:
import pandas as pd
import polars as pl
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit, StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer

import lightgbm as lgb

import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances, plot_slice, plot_contour

from time import time

import pickle

In [ ]:
# !gsutil cp /home/eanegrin/buckets/b1/datasets/competencia_02_fe_v01.parquet /home/eanegrin/datasets/

In [2]:
# base_path = '/content/drive/MyDrive/DMEyF/2024/'
base_path = 'C:/Eugenio/Maestria/DMEyF/'
# base_path = '/home/eanegrin/buckets/b1/'

dataset_path = base_path + 'datasets/'
modelos_path = base_path + 'modelos/'
db_path = base_path + 'db/'
dataset_file = 'competencia_02_fe_v01_undersampled_10.parquet'

ganancia_acierto = 273000
costo_estimulo = 7000

# agregue sus semillas
semillas = [122219, 109279, 400391, 401537, 999961]

# data = pd.read_parquet('/home/eanegrin/datasets/' + dataset_file)
data = pd.read_parquet(dataset_path + dataset_file)

In [3]:
meses_train = [202006,202007,202008,202009,202010,202011,202012,
               202101,202102,202103,202104,202105] # dejo afuera 202106 para test

data = data[data['foto_mes'].isin(meses_train)]
data.shape

(209841, 679)

In [4]:
# Asignamos pesos a las clases

data['clase_peso'] = 1.0

data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

In [5]:
X_train = data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria'], axis=1)
y_train_binaria = data['clase_binaria'] # Junta a los 2 baja
w_train = data['clase_peso']

Para evaluar la calidad del modelo, crearemos nuestra propia función de evaluación que calcule la ganancia. La razón de incluir los pesos es precisamente para poder implementar esta función de evaluación de manera adecuada. Al combinar las clases *BAJA+1* y *BAJA+2* en una sola, necesitamos una forma de diferenciarlas, y es aquí donde entra en juego el *weight*. Este parámetro nos permitirá distinguir entre ambas clases al momento de evaluarlas dentro del algoritmo.


In [6]:
def lgb_gan_eval(y_pred, data):
    weight = data.get_weight()
    ganancia = np.where(weight == 1.00002, ganancia_acierto, 0) - np.where(weight < 1.00002, costo_estimulo, 0)
    ganancia = ganancia[np.argsort(y_pred)[::-1]]
    ganancia = np.cumsum(ganancia)

    return 'gan_eval', np.max(ganancia) , True

# Optimizacion

In [7]:
def objective(trial):

    num_leaves = trial.suggest_int('num_leaves', 8, 150), # segun statquest esto deberia ir de 8 a 32
    learning_rate = trial.suggest_float('learning_rate', 0.005, 0.05), # mas bajo, más iteraciones necesita
    min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 1, 2000),
    feature_fraction = trial.suggest_float('feature_fraction', 0.1, 1.0),
    bagging_fraction = trial.suggest_float('bagging_fraction', 0.1, 1.0),

    params = {
        'objective': 'binary',
        'metric': 'custom',
        'boosting_type': 'gbdt',
        'first_metric_only': True,
        'boost_from_average': True,
        'feature_pre_filter': False,
        'max_bin': 31,
        'num_leaves': num_leaves,
        'learning_rate': learning_rate,
        'min_data_in_leaf': min_data_in_leaf,
        'feature_fraction': feature_fraction,
        'bagging_fraction': bagging_fraction,
        'seed': semillas[0],
        'verbose': -1
    }
    
    train_data = lgb.Dataset(X_train,
                              label=y_train_binaria, # eligir la clase
                              weight=w_train)
    
    # print(f"Learning Rate: {learning_rate}, Type: {type(learning_rate)}")
    
    # Use callbacks for early stopping
    early_stopping_cb = lgb.early_stopping(stopping_rounds=50) # creo que en min_delta le tendria que pasar un parametro con la ganancia que considero irrelevante
        
    cv_results = lgb.cv(
        params,
        train_data,
        num_boost_round=1000, # modificar, subit y subir... y descomentar la línea inferior (ahora le puso 100 para mostrarnos, pero hay que ponerle un numero alto, 10.000, 200.000)
        callbacks=[early_stopping_cb],
        feval=lgb_gan_eval,
        stratified=True,
        nfold=5,
        seed=semillas[0]
    )
    
    max_gan = max(cv_results['valid gan_eval-mean'])
    best_iter = cv_results['valid gan_eval-mean'].index(max_gan) + 1

    # Guardamos cual es la mejor iteración del modelo
    trial.set_user_attr("best_iter", best_iter)

    return max_gan * 5 # funcion objetivo, en el proximo paso le digo si quiero maximizarla o minimizarla.

In [8]:
storage_name = "sqlite:///" + db_path + "optimization_lgbm.db"
study_name = "competencia2_lgbm_v03" # UPDATE

study = optuna.create_study(
    direction="maximize",
    study_name=study_name,
    storage=storage_name,
    load_if_exists=True,
)

[I 2024-11-15 21:00:03,023] A new study created in RDB with name: competencia2_lgbm_v03


In [9]:
study.optimize(objective, n_trials=100)

Training until validation scores don't improve for 50 rounds


[I 2024-11-15 21:09:44,052] Trial 0 finished with value: 1871975000.0 and parameters: {'num_leaves': 23, 'learning_rate': 0.022749192879784674, 'min_data_in_leaf': 1342, 'feature_fraction': 0.696028516980907, 'bagging_fraction': 0.8119408210963277}. Best is trial 0 with value: 1871975000.0.


Did not meet early stopping. Best iteration is:
[994]	cv_agg's valid gan_eval: 3.74395e+08 + 8.9217e+06
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[407]	cv_agg's valid gan_eval: 3.78896e+08 + 1.08452e+07


[I 2024-11-15 21:11:46,091] Trial 1 finished with value: 1894480000.0 and parameters: {'num_leaves': 104, 'learning_rate': 0.046979208522369274, 'min_data_in_leaf': 1050, 'feature_fraction': 0.7098737760075081, 'bagging_fraction': 0.25783666439001973}. Best is trial 1 with value: 1894480000.0.


Training until validation scores don't improve for 50 rounds


...

[I 2024-11-16 01:16:22,009] Trial 97 finished with value: 1902586000.0 and parameters: {'num_leaves': 113, 'learning_rate': 0.04143152364309648, 'min_data_in_leaf': 366, 'feature_fraction': 0.4526488763257703, 'bagging_fraction': 0.12918850109782112}. Best is trial 21 with value: 1915501000.0.


Training until validation scores don't improve for 50 rounds


[I 2024-11-16 01:17:32,643] Trial 98 finished with value: 1900773000.0 and parameters: {'num_leaves': 138, 'learning_rate': 0.04079749643312253, 'min_data_in_leaf': 416, 'feature_fraction': 0.20441362819632497, 'bagging_fraction': 0.15591906788302054}. Best is trial 21 with value: 1915501000.0.


Early stopping, best iteration is:
[402]	cv_agg's valid gan_eval: 3.80155e+08 + 1.007e+07
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[715]	cv_agg's valid gan_eval: 3.81746e+08 + 8.43882e+06


[I 2024-11-16 01:20:03,608] Trial 99 finished with value: 1908732000.0 and parameters: {'num_leaves': 148, 'learning_rate': 0.03752325354173981, 'min_data_in_leaf': 181, 'feature_fraction': 0.43637567342502104, 'bagging_fraction': 0.2431066070682315}. Best is trial 21 with value: 1915501000.0.


Analizamos los resultados as usual

In [10]:
optuna.visualization.plot_optimization_history(study)

In [11]:
plot_param_importances(study)

El **learning rate** es un parámetro que tiene que ir acompañado por más árboles.

In [12]:
plot_slice(study)

In [13]:
plot_contour(study)

In [14]:
plot_contour(study, params=['num_leaves','min_data_in_leaf'])

In [15]:
study.best_trial.params

{'num_leaves': 150,
 'learning_rate': 0.03689843027994712,
 'min_data_in_leaf': 140,
 'feature_fraction': 0.4423192311397083,
 'bagging_fraction': 0.19127549734404992}

In [16]:
best_iter = study.best_trial.user_attrs["best_iter"]
best_iter

985